In [3]:
from models import *

# ACTIVATE GPU USAGE
 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'        
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

2023-05-16 10:54:42.442908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 10:54:42.934069: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/lib/:/home/hmorales/miniconda3/envs/twinnet/lib/
2023-05-16 10:54:42.934137: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No su

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


2023-05-16 10:54:43.983764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:54:43.983912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:54:43.984037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:54:44.410680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:54:44.410871: I tensorflow/compiler/xla/stream_executo

# TRAINING PROCESS - 2D MODEL


In [ ]:
# Define paths

modeltype = '2d'
src_path_Source = '/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/trainingImages/'
src_path_Target = '/media/hmorales/Skynet/MemNuclei2Nuclei/Nuclei/trainingImages/'


dataset = generate_datasets(model_type = modeltype,
                            images_folder_source = src_path_Source,
                            images_folder_target = src_path_Target,
                            images_extension= 'tif',
                            patch_shape = (128,128),
                            patch_step = 128,
                            threshold = 0.01,
                            percentiles = (40.0, 99.9999),
                            n_samples = 50000,
                            seed_a = 27,
                            seed_b = 57,
                            shuffle_ = False)               

In [ ]:
# plot some patches
Npairs = 6
figWidth=3 
imagesShape = dataset[0].shape    
maxPlanes = imagesShape[3] - 1  
nImages = imagesShape[0]

idxpatch=np.linspace(0, nImages-1, num=Npairs, dtype=np.int16);
idxpatch=np.random.randint(0,nImages,Npairs)
#print(idxpatch)

images1 = dataset[0]
images2 = dataset[1]

print(images1.shape)
def show(ax, image):
    ax.imshow(image, cmap='viridis', vmin=-1, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

   
fig = plt.figure(figsize=(2*figWidth,Npairs*figWidth))
axs = fig.subplots(Npairs,2)
for i in range(Npairs):
    if modeltype == '2d':
        show(axs[i,0], images1[idxpatch[i],:,:])
        show(axs[i,1], images2[idxpatch[i],:,:])
    elif modeltype == '3d':
        show(axs[i,0], np.amax(images1[idxpatch[i],:,:,:],2))
        show(axs[i,1], np.amax(images2[idxpatch[i],:,:,:],2))

    

plt.subplots_adjust(wspace=0, hspace=0)            
plt.show()

#tiff.imwrite('temp1.tif', images1)
#tiff.imwrite('temp2.tif', images2)

In [ ]:
# Train the model
outdir = ''
training = train_model_2D(dataset = dataset,
                          batch_size = 1, 
                          epochs = 100, 
                          save_freq = 2,
                          output_dir = outdir) 

In [4]:
# Predict test image
input_image_path='/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/testImages/G2_TL000_Angle90.tif'
output_name = '/media/hmorales/Skynet/MemNuclei2Nuclei/MemNuclei/testImages/Nuclei_G2_TL000_Angle90.tif'
model_path = '/home/hmorales/WorkSpace/Development/CycleGAN3D/StyleTransferMicroscopyImages-main/2DModel/models/g_model_AtoB_2D_114400.h5'
patch_size = 128
overlap_pct = 0
percentiles = (40.0, 99.9999)

prediction = predict_2D(input_image_path,  # path to image
                        percentiles, # for image normalization
                        output_name, # output filename. Will be saved in the working directory.
                        model_path, # path to model (.h5 file)
                        patch_size) # size of patch (single value)
 

2023-05-16 10:55:02.434788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:55:02.435113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:55:02.435326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:55:02.435599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 10:55:02.435815: I tensorflow/compiler/xla/stream_executo

Intensity from (0 , 6972) to  (0, 1389)  



100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.13s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.24s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.23s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.23s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:26<00:00,  3.27s/it]



image 268 of 268


100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.24s/it]


execution time is:  1:55:55.406442
Intensity from (0 , 65535) to  (0, 51475)  

